In [1]:
from scipy.spatial.transform import Rotation as R
import numpy as np
import warnings
import pandas as pd

In [2]:
file = open("psi4/glycylglycine.dat","r" )

In [3]:
print(file.read())


    -----------------------------------------------------------------------
          Psi4: An Open-Source Ab Initio Electronic Structure Package
                               Psi4 1.2 release

                         Git: Rev {HEAD} 9d4a61c 


    R. M. Parrish, L. A. Burns, D. G. A. Smith, A. C. Simmonett,
    A. E. DePrince III, E. G. Hohenstein, U. Bozkaya, A. Yu. Sokolov,
    R. Di Remigio, R. M. Richard, J. F. Gonthier, A. M. James,
    H. R. McAlexander, A. Kumar, M. Saitow, X. Wang, B. P. Pritchard,
    P. Verma, H. F. Schaefer III, K. Patkowski, R. A. King, E. F. Valeev,
    F. A. Evangelista, J. M. Turney, T. D. Crawford, and C. D. Sherrill,
    J. Chem. Theory Comput. 13(7) pp 3185--3197 (2017).
    (doi: 10.1021/acs.jctc.7b00174)


                         Additional Contributions by
    P. Kraus, H. Kruse, M. H. Lechner, M. C. Schieber, and R. A. Shaw

    -----------------------------------------------------------------------


    Psi4 started on: Wednesday, 29 Januar

In [6]:
file_csv = open("psi4/glycylglycine.csv","w")
with open("psi4/glycylglycine.dat","r") as filehandle:
    line_2 =9999999
    line_number = 0
    for line in filehandle:
        if 'Center' in line:
            line_2 = line_number
            print('In line_2', line_number)
        if 'Running in' in line:
            break
        elif line_number > line_2 + 1:
            file_csv.write(line) 
        line_number += 1

        
data = pd.read_csv("psi4/glycylglycine.csv", header=None, delim_whitespace=True)
data.columns = ["Element", "X", "Y", "Z", "Mass"]
print(data.shape)
print(type(data))
        

In line_2 87
(17, 5)
<class 'pandas.core.frame.DataFrame'>


In [7]:
class atom():
    def __init__(self,element,x,y,z,mass,aminoacid = None ,c_type = None,linked_to = [],vanderWals_r = 0):
        self.element = element
        self.x = x
        self.y = y
        self.z = z
        self.mass = mass
        self.aminoacid = aminoacid
        self.c_type = c_type
        self.linked_to = linked_to
        self.vanderWalls_r = 0
        
    def rotate(self,atom1,atom2,angle,angle_tpye): # Angle should be a fraction since it will be multiplied by 2pi
        
        # Check whether the angle is between the right atoms. Else raise exception
        if angle_type == 'phi':
            if atom1.c_type != N or atom2.c_type != C_alpha:
                raise Excepcion('Not the correct angle between N and C_alpha. The atoms are ',atom1.c_type,' and ',atom2.c_type)
        if angle_type == 'psi':
            if atom1.c_type != C_alpha or atom2.c_type != Carboxy:
                raise Excepcion('Not the correct angle between C_alpha and Carboxy. The atoms are ',atom1.c_type,' and ',atom2.c_type)
        else: 
            raise Exception('The angle is not phi or psi!')
            
        v1 = np.array([atom1.x,atom1.y,atom1.z])
        v2 = np.array([atom2.x,atom2.y,atom2.z])
        v = v2-v1
        if np.linalg.norm(v) < 0.001:
            warnings.warn('Are you sure atom1 and atom2 are different? They seem to have the same position')
        v /= np.linalg.norm(v)
        
        r = R.from_rotvec(2*np.pi*angle * np.array([0, 0, 1]))
        M = r.as_matrix()
        
        #Firs thing we set a new coordinate system where v1 = [0,0,0] (translation only) so that we can perform the rotation of M
        p = np.array([self.x,self.y,self.z])
        p -= v1
        
        
        #Next we perform the rotation along the axis 
        p = M@p
        
        #Undo the translation
        p += v1
        self.x = p[0]
        self.y = p[1]
        self.z = p[2]
        

list_of_atoms = []
print(list_of_atoms)
for row in range(data.shape[0]):
    line = data.loc[row, :]
    list_of_atoms += [atom(line[0],line[1],line[2],line[3],line[4])]
    
for at in list_of_atoms:
    print(at.element,at.c_type)
    print(at.x,at.y,at.z)
    print(at.linked_to)

[]
O None
1.126249653099 -0.9920575895229999 0.915225154709
[]
O None
-1.690950346901 1.220642410477 0.570525154709
[]
O None
-3.289150346901 -0.348957589523 0.21062515470899998
[]
N None
0.126449653099 0.055142410477 -0.9006748452910001
[]
N None
3.565449653099 0.005642410477 0.144025154709
[]
C None
-1.143450346901 -0.633057589523 -0.842874845291
[]
C None
1.162849653099 -0.18385758952300002 -0.010074845291
[]
C None
2.3610496530989997 0.700742410477 -0.287174845291
[]
C None
-2.1549503469010003 0.073642410477 0.022425154709
[]
H None
-0.988250346901 -1.640357589523 -0.445174845291
[]
H None
-1.5312503469010001 -0.684257589523 -1.863974845291
[]
H None
0.239249653099 0.769142410477 -1.614374845291
[]
H None
2.242049653099 1.6372424104769998 0.266025154709
[]
H None
2.4374496530989997 0.933042410477 -1.353674845291
[]
H None
3.6516496530989997 -0.880057589523 -0.352974845291
[]
H None
3.4982496530989997 -0.223757589523 1.134825154709
[]
H None
-2.3672503469010002 1.6612424104769998 1.

Link atoms

In [8]:
def distance(atom,atom2):
    return np.sqrt((atom.x-atom2.x)**2+(atom.y-atom2.y)**2+(atom.z-atom2.z)**2)
print(list_of_atoms)

[<__main__.atom object at 0x128ab95c0>, <__main__.atom object at 0x128ab9668>, <__main__.atom object at 0x128ab96d8>, <__main__.atom object at 0x128ab9710>, <__main__.atom object at 0x128ab9748>, <__main__.atom object at 0x128ab9780>, <__main__.atom object at 0x128ab97f0>, <__main__.atom object at 0x128ab9828>, <__main__.atom object at 0x128ab9860>, <__main__.atom object at 0x128ab9898>, <__main__.atom object at 0x128ab98d0>, <__main__.atom object at 0x128ab9908>, <__main__.atom object at 0x128ab9940>, <__main__.atom object at 0x128ab9978>, <__main__.atom object at 0x128ab99b0>, <__main__.atom object at 0x128ab99e8>, <__main__.atom object at 0x128ab9a20>]


Calculate the links for each atom. As a choice we make an hidrogen connected to an atom if the distance is smaller than 1.3 amstrongs and distance between other atoms smaller than 2 amstrongs. 

In [13]:
for at in list_of_atoms:
    at.linked_to = []

for at1 in list_of_atoms:
    for at2 in list_of_atoms:
        if at1 != at2:
            if at1.element == 'O' and distance(at1,at2)<2 and at2.element == 'C':
                at2.c_type = 'Carboxy'
                at1.linked_to = [at2] + at1.linked_to 
                at2.linked_to = [at1] + at2.linked_to
                
            if at1.element == 'N' and distance(at1,at2)<2 and at2.element == 'C':
                if at2.c_type != 'Carboxy':
                    at2.c_type = 'C_alpha'
                at1.linked_to = [at2] + at1.linked_to 
                at2.linked_to = [at1] + at2.linked_to
                
            if at1.element == 'H' and distance(at1,at2)<1.3 and at2.element == 'C': 
                at1.linked_to = [at2] + at1.linked_to 
                at2.linked_to = [at1] + at2.linked_to
                
            if at1.element == 'C' and distance(at1,at2)<2 and at2.element == 'C' and (at1 not in at2.linked_to) : 
                at1.linked_to = [at2] + at1.linked_to 
                at2.linked_to = [at1] + at2.linked_to 

for at1 in list_of_atoms:
    print('new item',at1.element)
    for at in at1.linked_to:
        print(at.element)

1
[]
[]
<class 'list'> <class 'list'>
2
[]
[]
<class 'list'> <class 'list'>
3
[]
<class 'list'> <class 'list'>
[<__main__.atom object at 0x128ab96d8>, <__main__.atom object at 0x128ab9668>]
new item O
C
new item O
C
new item O
C
new item N
C
C
new item N
C
new item C
H
H
C
N
new item C
C
N
O
new item C
H
H
C
N
new item C
C
O
O
new item H
C
new item H
C
new item H
new item H
C
new item H
C
new item H
new item H
new item H


In [ ]:
print(list_of_atoms)
for at1 in list_of_atoms:
    print(at.linked_to)
'''
for at1 in list_of_atoms:
    for at2 in at1.linked_to:
        print(at2.element)
'''


In [ ]:
'''
for atom in list_of_atoms:
    for atom2 in list_of_atoms:
        if distance(atom,atom2)<1.8:
            print(atom.element,atom2.element,distance(atom,atom2))


for atom in list_of_atoms:
    if atom.element == 'O':
        for atom2 in list_of_atoms:
            #print(atom.linked_to)
            #[print(atom2.element) for atom2 in atom.linked_to]
            if atom2.element == 'C' and (distance(atom,atom2) < 2) and (distance(atom,atom2) != 0) and (atom2 not in atom.linked_to):
                print('distance: ',distance(atom,atom2))
                atom2.c_type = 'Carboxy'
                atom2.linked_to += [atom]
                atom.linked_to += [atom2]
    
    elif atom.element == 'N':
        atom.c_type = 'N'
        for atom2 in list_of_atoms:
            if atom2.element == 'C' and (distance(atom,atom2) < 2) and (distance(atom,atom2) != 0) and atom2 not in atom.linked_to:
                print('distance: ',distance(atom,atom2),'2')
                atom2.linked_to += [atom]
                atom.linked_to += [atom2]
                
                if atom2.c_type != 'Carboxy':
                    atom2.c_type = 'C_alpha'
                    
    elif atom.element == 'H':
        for atom2 in list_of_atoms:
            if distance(atom,atom2) < 1.3 and (distance(atom,atom2) != 0) and atom2 not in atom.linked_to:
                print('distance: ',distance(atom,atom2),'3')
                atom.linked_to += [atom2]
                atom2.linked_to += [atom]
'''
'''
print(list_of_atoms[0].linked_to)
for atom in list_of_atoms:
    print(atom.element,atom.x,atom.y,atom.z,atom.mass,atom.c_type)
    print(atom.element,atom.c_type,' linked to ',[atom2.element for atom2 in atom.linked_to])
'''

In [ ]:
class aminoacid():
    def __init__(self,residue,phi,psi,atom_list = [],C_alpha = None, N= None, Carboxy = None, position = None, Radical = None):
        self.residue = residue
        self.atom_list = atom_list
        self.phi = phi
        self.psi = psi
        self.C_alpha = C_alpha
        self.N = N
        self.Carboxy = Carboxy
        self.position = position
        self.Radical = Radical
        

        


In [ ]:
def backbone_to_rotate(angle_type,starting_atom):
    # Define the list of atoms to rotate and then rotate them
    list_of_atoms_to_rotate = []
    if angle_type == 'phi': # Follows the structure N -> Carboxy -> C_alpha -> N
        for atom in starting_atom.linked_to:
            if starting_atom.c_type == 'N' and atom.c_type == 'Carboxy':
                list_of_atoms_to_rotate += [atom]
                list_of_atoms_to_rotate += backbone_to_rotate(angle_type = angle_type, starting_atom = atom)

            elif starting_atom.c_type == 'Carboxy' and atom.c_type == 'C_alpha':
                list_of_atoms_to_rotate += [atom]
                list_of_atoms_to_rotate += backbone_to_rotate(angle_type = angle_type, starting_atom = atom)

            elif starting_atom.c_type == 'C_alpha' and atom.c_type == 'N':
                list_of_atoms_to_rotate += [atom]
                list_of_atoms_to_rotate += backbone_to_rotate(angle_type = angle_type, starting_atom = atom)

            #elif atom.c_type == None:
            #list_of_atoms_to_rotate += additional_atoms()
            ## Leave this until we have backbone

    elif angle_type == 'psi': # Follows the structure N -> C_alpha -> Carboxy -> N
        for atom in starting_atom.linked_to:
            if starting_atom.c_type == 'Carboxy' and atom.c_type == 'N':
                list_of_atoms_to_rotate += [atom]
                list_of_atoms_to_rotate += backbone_to_rotate(angle_type = angle_type, starting_atom = atom)

            elif starting_atom.c_type == 'C_alpha' and atom.c_type == 'Carboxy':
                list_of_atoms_to_rotate += [atom]
                list_of_atoms_to_rotate += backbone_to_rotate(angle_type = angle_type, starting_atom = atom)

            elif starting_atom.c_type == 'N' and atom.c_type == 'C_alpha':
                list_of_atoms_to_rotate += [atom]
                list_of_atoms_to_rotate += backbone_to_rotate(angle_type = angle_type, starting_atom = atom)            
    else: 
        raise Exception('Angle_type should be either phi or psi. However it currently is {}'.format(angle_type))
        
    return list_of_atoms_to_rotate


def decorations_to_rotate(backbone_list):
    '''Input: list of backbone atoms. Returns a list of other atoms that must be rotated'''
    list_of_atoms_to_rotate = []
    addition_list = []
    for back_atom in backbone_list:
        for atom in back_atom.linked:
            if atom.c_type = None and atom not in list_of_atoms_to_rotate:
                addition_list += [atom]
                list_of_atoms_to_rotate += [atom]

        while addition_list != []:
            old_list = addition_list
            addition_list = []
            for prev_atom in old_list:
                for atom in prev_atom.linked:
                    if atom.c_type = None and atom not in list_of_atoms_to_rotate:
                        addition_list += [atom]
                        list_of_atoms_to_rotate += [atom]

    return list_of_atoms_to_rotate


def rotate(angle_type,starting_atom):
    if angle_type == 'phi':
        if starting_atom.c_type != 'N':
            raise Exception('Wrong starting atom for the angle phi:',starting_atom.c_type )
        for atom in starting_atom.linked_to:
            if atom.c_type = 'C_alpha':
                atom_c_alpha = atom
                
    elif angle_type == 'psi':
        if starting_atom.c_type != 'Carboxy':
            raise Exception('Wrong starting atom for the angle phi:',starting_atom.c_type )
        for atom in starting_atom.linked_to:
            if atom.c_type = 'C_alpha':
                atom_c_alpha = atom
    else:
        raise Exception('Angle not recognised!:',angle_type)
    
    # Define the list of atoms to rotate and then rotate them
    
    list_of_atoms_to_rotate = []
    list_of_atoms_to_rotate += backbone_to_rotate(angle_type,starting_atom)
    list_of_atoms_to_rotate += decorations_to_rotate(list_of_atoms_to_rotate)
    
    for atom in list_of_atoms_to_rotate:
        # The rotation angle is pi/16 and in order to maintain internal coherence 
        atom.rotate(atom_c_alpha,starting_atom,angle = 1/32,angle_type = angle_type)  

We should probably plot first the protein so that we can stablish the links correctly

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

def randrange(n, vmin, vmax):
    '''
    Helper function to make an array of random numbers having shape (n, )
    with each number distributed Uniform(vmin, vmax).
    '''
    return (vmax - vmin)*np.random.rand(n) + vmin

n=10


for c, m, zlow, zhigh in [('r', 'o', -50, -25), ('b', '^', -30, -5)]:
    xs = randrange(n, 23, 32)
    ys = randrange(n, 0, 100)
    zs = randrange(n, zlow, zhigh)
    ax.scatter(xs, ys, zs, c=c, marker=m)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [ ]:
xs = [0,1]
ys = [0,1]
Axes3D.scatter(xs, ys, zs=0, zdir='z', s=20, c=None, depthshade=True)

Here we experiment how to perform rotations

In [ ]:
r = R.from_rotvec(np.pi/2 * np.array([0, 0, 2]))
r.as_matrix()

In [ ]:
v = np.array([0,0,0])
r.as_matrix()@v